
### Hello,

---



### **Before we began please upload the following files (They are similar to the files that are attached to the email) :**

- dev.json
- tables.json
- database.zip ( this zip file contain a database folder)
- predict.txt ( this file contain the prediction queries from Star or Rasat model )

### **In Cell 1 please do the following**

Fill in the openai.organization = "org-XXX" **( write your own organization key instead of org-XXX)**

Fill in the openai.api_key = 'sk-xxxxx' **( write your own Open AI key instead of sk-xxxxx )**

Fill in the 'Authorization': f'Bearer sk-xxxxx' **( write your own Open AI key instead of sk-xxxxx )**

---



### **After you done from writing the keys, please run Cell 1, 2, 3, 4, 5, 6 in the same order**

Cell 6: is the one that going to call GPT-3.5-turbo and only going to take approximatly 20 min for 1000 questions

---



### **Once Cell 6 ends run Cell 7 and you will get the new_predict.txt file**

# **Cell 1**

In [ ]:
import json
import sqlite3
import os
import requests
import re
!pip3 install openai==0.27.0
import os
import openai
import time
import re
import time
import numpy as np
!pip install sentence_transformers

openai.organization = "org-XXXXXXXXXXX"
openai.api_key = 'sk-XXXXXXXXXXX'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer sk-XXXXXXXXXXX'
}

openai.Model.list()

with open('/content/dev.json', encoding='utf-8') as o:
        dev = json.load(o)

with open('/content/tables.json', encoding='utf-8') as o:
        tables = json.load(o)

!unzip /content/database.zip -d /content/

# **Cell 2**

In [ ]:
def get_DB_values(db_id):
  path = '/content/database/' + db_id + '/' + db_id + '.sqlite'
  sqlite_db_path = path
  conn = connect_sqlite(sqlite_db_path)
  first_eight_rows_data_with_columns = fetch_first_eight_rows_with_column_names(conn)
  return first_eight_rows_data_with_columns


# Define the function to connect to the SQLite database file
def connect_sqlite(db_path):
    connection = sqlite3.connect(db_path)
    return connection

# Define the function to generate SQL for creating tables
def generate_create_table_sql(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    create_table_sql = []

    for table_name in tables:
        cursor.execute(f"PRAGMA table_info({table_name[0]})")
        columns = cursor.fetchall()

        column_definitions = []
        for column in columns:
            col_def = f"{column[1]} {column[2]}"
            if column[3]:  # Checking for NOT NULL
                col_def += " NOT NULL"
            if column[5]:  # Checking for primary key
                col_def += " PRIMARY KEY"
            column_definitions.append(col_def)

        create_stmt = f"CREATE TABLE {table_name[0]} ({', '.join(column_definitions)});"
        create_table_sql.append(create_stmt)

    return create_table_sql

# Define the function to fetch the first two rows and column names from each table
def fetch_first_eight_rows_with_column_names(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    table_data_with_columns = {}

    for table_name in tables:
        cursor.execute(f"SELECT * FROM {table_name[0]} LIMIT 8;")
        data = cursor.fetchall()
        cursor.execute(f"PRAGMA table_info({table_name[0]})")
        columns = [col[1] for col in cursor.fetchall()]  # Extract column names
        table_data_with_columns[table_name[0]] = {'columns': columns, 'rows': data}

    return table_data_with_columns

# **Cell 3**

In [ ]:
## GET the model predictions from RASAT MODEL or STAR MODEL

with open('/content/predict.txt', 'r') as file:
    # Reading the content of the file
    file_content = file.read()

with open('/content/test.json', encoding='utf-8') as o:
        dev = json.load(o)

# Split the content by new lines into an array
sql_queries_array_from_file = file_content.split('\n')
cleaned_list = [s for s in sql_queries_array_from_file if s.strip() != ""]

ArrayModel_prediction = []
i = 0
for e in dev:
  interaction_list = []
  for s in e['interaction']:
    tempdict = {}
    tempdict['db_id'] = e['database_id']
    tempdict['question'] = s['utterance']
    tempdict['query'] = cleaned_list[i]
    i = i + 1
    interaction_list.append(tempdict)

  ArrayModel_prediction.append(interaction_list)


# Now we will save this array to a JSON file.
json_file_path = 'Model_prediction.json'

# Writing the array to a JSON file.
with open(json_file_path, 'w') as json_file:
    json.dump(ArrayModel_prediction, json_file, indent=4)

# **Cell 4**

In [ ]:
def getModelPrediction(interactionCounter ,sampleInInteractionCounter):
  with open(json_file_path, encoding='utf-8') as o:
        dev = json.load(o)

  prediction = dev[interactionCounter][sampleInInteractionCounter]

  return prediction

# **Cell 5**

In [ ]:
def OpenAI_Prompt(question, schema, FirstQuestion, previous_query , questionDB, with_later_prediction, interactionCounter ,sampleInInteractionCounter):

    def schema_Extraction_for_OpenAI_prompt(schema):
            schemaString =  ""
            i = 0
            for t in schema['table_names_original']:
              schemaString =  schemaString + "# Table " + t + ", columns = [ "
              for c in schema['column_names_original']:
                if c[0] == i:
                  schemaString = schemaString + c[1] + ", "

              schemaString = schemaString[:-2] + " ]"
              schemaString = schemaString + " \n"
              i = i + 1
            schemaString = schemaString + "# Primary_keys = [ "
            for pk in schema['primary_keys']:
              column_counter = 0
              for c in schema['column_names_original']:
                if column_counter == pk:
                  schemaString = schemaString + schema['table_names_original'][c[0]]+ '.' + c[1] + ", "
                column_counter = column_counter + 1
            schemaString = schemaString[:-2] + " ]"
            schemaString = schemaString + " \n"
            schemaString = schemaString + "# Foreign_keys = [ "
            for fk in schema['foreign_keys']:
              column_counter = 0
              for c in schema['column_names_original']:
                if column_counter == fk[0]:
                  schemaString = schemaString + schema['table_names_original'][c[0]]+ '.' + c[1] + " foreign key referenced to "
                  another_column_counter = 0
                  for c in schema['column_names_original']:
                    if another_column_counter == fk[1]:
                      schemaString = schemaString + schema['table_names_original'][c[0]]+ '.' + c[1] + " ], ["
                    another_column_counter = another_column_counter + 1
                column_counter = column_counter + 1
            schemaString = schemaString[:-3]
            return schemaString

    # Start creating the prompt
    # Construct the schema
    Intro = "### Complete sqlite SQL query only and with no explanation " + "\n" + "### SQLite SQL tables , with their properties :"
    Created_schema = schema_Extraction_for_OpenAI_prompt(schema)

    # Control the response style
    #instruction = "Answer the follwoing question: "
    Correcting_message = "# Please provide a direct SQL query with no additional text or explanations to correct it if necessary"
    Response_style = "# The response must be a complete SQL query in code snippet starting with SELECT and ending with a semicolon."
    Database_sentance = "# These are examples of the first eight rows of the database scehma"
    database_values = str(get_DB_values(schema['db_id']))
    Introduce_question = "# For the following question:"
    Introduce_prediction = "# Our model XSQL predict this query:"

    # Construct the Prompt
    prediction = getModelPrediction(interactionCounter ,sampleInInteractionCounter)
    prediction = prediction['query']


    if FirstQuestion:
      Prompt = Intro + " \n" + Created_schema + " \n" + Database_sentance + " \n" + '#' + database_values + " \n" + Introduce_question + "### " + question + " \n" + Introduce_prediction + " \n" + prediction + " \n" + Correcting_message + " \n" + Response_style
    elif FirstQuestion == False and with_later_prediction:
      T1 = "# Based on the previous query: "
      Prompt = T1 + previous_query + " \n" + "### " + question + " \n" + Introduce_prediction + " \n" + prediction + " \n" + Correcting_message + " \n" + Response_style
    elif FirstQuestion == False and with_later_prediction == False:
      T1 = "# Based on the previous query: "
      T2 = "# Please provide a direct SQL query with no additional text or explanations to answer this question:"
      Prompt = T1 + previous_query + " \n" + T2 + " \n" + "### " + question + " \n" + Response_style


    return Prompt

# **Cell 6**

In [ ]:
model = "gpt-3.5-turbo"
with_later_prediction = True

def cleanResponse(result):
  pattern = r"SELECT .*?;"
  matches = re.findall(pattern, result, re.IGNORECASE | re.DOTALL)
  for match in matches:
      cleaned_match = match.strip().replace('\n', ' ')
  return cleaned_match

GPT_final_result = []
counter = 0

interactionCounter = -1
for e in dev:
  interactionCounter = interactionCounter + 1
  counter = counter + 1
  # in case the model stop see the last number of the counter the model reach and write it instead of counter < 0
  if counter < 0:
    continue
  print(counter)

  interaction_result = []
  # Finded the needed schema
  for t in tables:
    if e['database_id'] == t['db_id']:
      schema = t
      break

  # Give the GPT4 or GPT3.5 a role name
  messages = [
        {"role": "system", "content": "You are a helpful assistant that provides SQL queries based on user questions"},
      ]
  # working first with the first question in the interaction
  FirstQuestion = True
  sampleInInteractionCounter = -1
  for u in e['interaction']:
      sampleInInteractionCounter = sampleInInteractionCounter + 1
      if FirstQuestion == True:
        previous_query = ""
        Prompt = OpenAI_Prompt(u['utterance'], schema, FirstQuestion, previous_query, e['database_id'], with_later_prediction, interactionCounter ,sampleInInteractionCounter)
        #print(Prompt)
        #print("---")
        FirstQuestion = False
      else:
        previous_query = sql_query
        Prompt = OpenAI_Prompt(u['utterance'], schema, FirstQuestion, previous_query, e['database_id'], with_later_prediction, interactionCounter ,sampleInInteractionCounter)
        #print(Prompt)
        #print("---")

      messages.append({"role": "user", "content": Prompt})
      retry_count = 0
      max_retries = 5
      response = None
      while retry_count < max_retries:
          try:
              response = openai.ChatCompletion.create(
                  model=model,
                  messages=messages,
                  temperature= 0,
                  request_timeout=32
              )
              break  # Break the retry loop if successful

          except:
              retry_count += 1
              print(f"Retry attempt {retry_count}/{max_retries}")
              time.sleep(5)  # Wait for a while before retrying

      if response is None:
          print("Max retries reached. Skipping this query.")
          continue  # Skip this query if max retries are reached

      # Get the results of the content
      result = response.choices[0].message['content']
      try:
      # clear the prompt to make sure that only include the sql query
        sql_query = cleanResponse(result)
      except:
        sql_query = "SELECT * FROM ERROR"

      #print("Predicted query ", sql_query)
      #print("*****************************************")
      messages.append({"role": "assistant", "content": sql_query})
      interaction_result.append(sql_query)
      #print(interaction_result)

  GPT_final_result.append(interaction_result)
  #print("----------------- ----------------- ----------------- -----------------")


FileName = "output.json"

with open(FileName, 'wt') as samozen:
   json.dump(GPT_final_result, samozen, sort_keys=False, indent=4, separators=(',', ': '))

# **Cell 7**

### Run this cell to get the final new_pridct.txt file , that can be used to get the results

In [ ]:
#generate the new_predict.txt file

with open(FileName, encoding='utf-8') as o:
        predict = json.load(o)


with open("new_predict.txt", "w") as file:
    # Iterate through each element in the array
    for element in predict:
        for sample in element:
        # Write each element as a new line in the text file
          file.write(sample + "\n")
        file.write("\n")

print("File has been created and data has been written!")

# **Thank you very much for your help**